Before running, follow the installation instructions in the `AdversarialInfluenceWorkbench/examples/microcredit_mixture/README.md` file.

In [1]:
import paragami
import vittles
import autograd
from autograd import numpy as np
from autograd import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

import time
import scipy as osp

In [2]:
import microcredit_mixture_rgiordandev
from microcredit_mixture_rgiordandev import microcredit_vb_lib as mm_lib

In [3]:
from microcredit_mixture_rgiordandev import \
    get_log_prob, get_param_pattern, get_log_prob_with_jacobian

In [4]:
np.random.seed(42)

## Load the data

In [5]:
raw_data = pd.read_csv('../R/data/microcredit_project_data_cleaned.csv') 
print(raw_data)
data = dict()

use_smuber = False

data['site'] = raw_data["site"].to_numpy().astype('int')
data['profit'] = raw_data["profit"].to_numpy()
data['treatment'] = raw_data["treatment"].to_numpy().astype('int')
data['cat'] = raw_data["cat"].to_numpy().astype('int')

# If use_smuber is False or unset, then use the log normal loss.
data['use_smuber'] = use_smuber

       site      profit  treatment  cat
0         1    0.000000          0    2
1         1  513.246492          1    3
2         1    0.000000          1    2
3         1    0.000000          0    2
4         1    0.000000          0    2
...     ...         ...        ...  ...
35298     7  -10.637987          0    1
35299     7   87.503514          0    3
35300     7   -5.736498          0    1
35301     7    0.000000          0    2
35302     7   28.722566          0    3

[35303 rows x 4 columns]


## Define priors and parameter structures
Note that I use the same parameterization as the model `tailored-hierarchical-pdf-log-normal.stan`, without the scaling.

In [6]:
prior_param_pattern = mm_lib.get_prior_param_pattern()
prior_params = mm_lib.get_default_prior_params()
prior_param_pattern.flatten(prior_params, free=False)

array([  0,   5,   0,   2,   0,   1,   0, 100,   0, 100,   0,   2,   0,
         2,   0, 100,   0,   2,   0, 100,   0,   2])

In [7]:
assert np.min(data['site']) == 1
num_sites = int(np.max(data['site'])) # K in stan.  Convert to int to allow json serialization.

param_pattern = get_param_pattern(num_sites)

In [8]:
params_rand = param_pattern.random()

In [9]:
# Test that these work
print(get_log_prob(params_rand, data, prior_params))
print(get_log_prob_with_jacobian(params_rand, data, prior_params, param_pattern))

-76783.3276739646
-76777.87533847814


## Define the ADVI parameters and objective.

I fit a non-standard version of ADVI where I fix a certain `num_draws` number of multivariate normal draws in advance.

Ideally, check afterwards that `num_draws` is large enough that the frequentist variance (due to the Monte Carlo integration) of the optimum  is small relative to the Bayesian posterior.

In [10]:
num_draws = 30

advi_param_pattern = mm_lib.get_advi_pattern(param_pattern)
base_draws = mm_lib.get_base_draws(num_draws, param_pattern)

advi_params = advi_param_pattern.random()
mm_lib.get_advi_loss(advi_params, base_draws, data, prior_params, param_pattern)

148220318.63768724

In [11]:
base_draws = mm_lib.get_base_draws(num_draws, param_pattern)
objective = mm_lib.get_advi_objective(
    base_draws, data, prior_params, advi_param_pattern, param_pattern)

advi_params_rand = advi_param_pattern.random()
advi_params_rand_free = advi_param_pattern.flatten(advi_params_rand)

## Run an initial fit.

In [12]:
init_advi_params = advi_param_pattern.empty(valid=True)
init_advi_params['mean'][:] = 0.0
init_advi_params['log_sd'][:] = -1

opt_init_time = time.time()
objective.reset()
objective.set_log_every(1)
opt_result = osp.optimize.minimize(
    method='trust-ncg',
    x0=advi_param_pattern.flatten(init_advi_params, free=True),
    fun=objective.f,
    jac=objective.grad,
    hessp=objective.hessian_vector_product,
    options={'maxiter': 100})
opt_init_time = time.time() - opt_init_time
print(opt_init_time)

Iter 0: f = 237889.26613274
Iter 1: f = 144444.13324077
Iter 2: f = 93334.88510482
Iter 3: f = 66735.36381635
Iter 4: f = 57913.86865819
Iter 5: f = 52161.76092000
Iter 6: f = 46783.53900470
Iter 7: f = 200683.06269075
Iter 8: f = 43657.42335466
Iter 9: f = 51040.59121014
Iter 10: f = 42468.64990199
Iter 11: f = 41092.10299056
Iter 12: f = 40714.07439810
Iter 13: f = 39124.39338311
Iter 14: f = 38587.41098704
Iter 15: f = 38106.23340968
Iter 16: f = 37749.56298242
Iter 17: f = 37490.78034736
Iter 18: f = 37341.10511961
Iter 19: f = 37072.16886996
Iter 20: f = 36917.04528396
Iter 21: f = 36691.81997166
Iter 22: f = 36640.72417442
Iter 23: f = 36541.10013801
Iter 24: f = 36452.59844152
Iter 25: f = 36385.86155779
Iter 26: f = 36290.12310610
Iter 27: f = 36169.81550724
Iter 28: f = 36096.21174713
Iter 29: f = 36015.64767551
Iter 30: f = 35916.88882083
Iter 31: f = 35891.49756736
Iter 32: f = 35826.57061853
Iter 33: f = 35821.80349081
Iter 34: f = 36134.89849299
Iter 35: f = 35805.70404262

## Preconditioned optimization

Evaluate the Hessian at the initial fit, check for positive definiteness, and then re-run with preconditioning to make sure we're at a good optimum.

In [13]:
hess_pc = objective.hessian(opt_result.x)

In [14]:
evs = np.linalg.eigvals(hess_pc)
np.min(evs), np.max(evs)

(0.3244618046610828, 20214.047484817587)

In [15]:
get_log_loss_free = paragami.FlattenFunctionInput(
    lambda advi_params: \
        mm_lib.get_advi_loss(advi_params, base_draws, data,
                             prior_params, param_pattern, weights=None),
    patterns=advi_param_pattern,
    free=True)

advi_loss_precond = paragami.PreconditionedFunction(get_log_loss_free)
advi_loss_precond.set_preconditioner_with_hessian(hessian=hess_pc)
objective_precond = paragami.OptimizationObjective(advi_loss_precond)
init_x = advi_loss_precond.precondition(opt_result.x)

In [16]:
opt_pc_time = time.time()
objective_precond.reset()
objective_precond.set_log_every(1)
opt_result = osp.optimize.minimize(
    method='trust-ncg',
    x0=init_x,
    fun=objective_precond.f,
    jac=objective_precond.grad,
    hessp=objective_precond.hessian_vector_product,
    options={'maxiter': 500})
print(opt_result.message)
opt_pc_time = time.time() - opt_pc_time
print(opt_pc_time)

Iter 0: f = 35738.44992764
Optimization terminated successfully.
1.163205623626709


In [17]:
# The total optimization time is the sum of both.
opt_time = opt_init_time + opt_pc_time

In [18]:
print(opt_time / 60)
print(opt_result.message)

24.928637476762137
Optimization terminated successfully.


## Inspect and save results.

In [19]:
advi_params = advi_param_pattern.fold(advi_loss_precond.unprecondition(opt_result.x))
advi_params_free = advi_param_pattern.flatten(advi_params, free=True)
params_opt = param_pattern.fold(advi_params['mean'])
#print(params_opt)

In [20]:
hess_time = time.time()
hess0 = objective.hessian(advi_params_free)
hess_time = time.time() - hess_time
evs = np.linalg.eigvals(hess0)

In [21]:
print(hess_time)
print(np.max(evs), np.min(evs), np.max(evs) / np.min(evs))

25.513100147247314
20214.047484814568 0.3244618046601927 62300.23748399182


In [22]:
grad0 = objective.grad(advi_params_free)
print(np.linalg.norm(grad0))
print(np.linalg.norm(np.linalg.solve(hess0, grad0)))

2.773361238178103e-06
2.1844107234763186e-06


In [56]:
save_dict = {}
save_dict['opt_time'] = opt_time
save_dict['hess_time'] = hess_time
save_dict['hess0'] = hess0
save_dict['advi_params_free'] = advi_param_pattern.flatten(advi_params, free=True)
save_dict['params_free'] = param_pattern.flatten(params_opt, free=True)
save_dict['advi_param_pattern_json'] = advi_param_pattern.to_json()
save_dict['param_pattern_json'] = param_pattern.to_json()
save_dict['base_draws'] = base_draws
save_dict['use_smuber'] = data['use_smuber']
#save_dict['optimization_log'] = objective_precond.optimization_log
save_dict['prior_param_pattern_json'] = prior_param_pattern.to_json()
save_dict['prior_params_flat'] = prior_param_pattern.flatten(prior_params, free=False)


In [57]:
out_filename = 'output/microcredit_project_advi_{}draws_smuber{}.npz'.format(
    base_draws.shape[1], data['use_smuber'])
print('Saving to {}'.format(out_filename))

np.savez_compressed(**save_dict, file=out_filename)

Saving to output/microcredit_project_advi_30draws_smuberFalse.npz
